### Trabalho 2 - Conway's Game of Life
###### Grupo 19

Tiago Passos Rodrigues - A96414

### Implementação

Definir o inputs do $N$ tamanho do campo de jogo, vetores constantes das bordas com $prob$ (probabilidade) das bordas serem 1, caso contrário 0.

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT
import random, numpy

N = 7 #tamanho da grelha

#centro = (random.randint(0,N),random.randint(0,N)) #random menos das bordas
centro = (3,3)
print(centro)
prob = 0.3 # probabilidade para o valor de 1 nas bordas

# criacao de 2 vetores costantes e uma constante da ponta da borda
zero_zero = numpy.random.choice([0,1], p=[1 - prob, prob])
vetorx = []
for i in range(N):
    vetorx.append(numpy.random.choice([0,1], p=[1 - prob, prob]))
vetory = []
for i in range(N):
    vetory.append(numpy.random.choice([0,1], p=[1 - prob, prob]))
    
'''
(x,y) = centro
    
if x == 0 and y == 0:
    zero_zero = 1
    #print("zero:",zero_zero)
    vetorx[0] = 1
    vetory[0] = 1
    vetorx[1] = 1
    vetory[1] = 1
elif x == 0:
    vetorx[x-1] = 1
    vetorx[x] = 1
    vetorx[x+1] = 1
'''
print("Valor do 0_0 = ", zero_zero)
print("Borda pelo linha do x = ", vetorx)
print("Borda pela linha do y = ", vetory)
# condicoes para os simbolos da grelha que estejam na ponta

(3, 3)
Valor do 0_0 =  0
Borda pelo linha do x =  [1, 0, 0, 0, 0, 0, 0]
Borda pela linha do y =  [0, 0, 1, 1, 1, 0, 0]


### Funções auxiliares

Criação das funções para as células, contagem da vizinhança, estado de inicio e transições de grelha para grelha.

In [2]:
# criacao do campo de jogo
def declare(i,j):
    return Symbol('i'+str(i)+'j'+str(j),INT)

# devolve o nº de vizinhos a 1
def vizinhanca(celula,grelha):
    x = celula[0]
    y = celula[1]   
    if y == N-1 and x == N-1:
        return (grelha[x-1][y] + grelha[x][y-1] + grelha[x-1][y-1])
    elif y == N-1:
        return (grelha[x+1][y] + grelha[x-1][y] + grelha[x][y-1] + grelha[x+1][y-1] + grelha[x-1][y-1])
    elif x == N-1:
        return (grelha[x-1][y] + grelha[x][y-1] + grelha[x][y+1] 
            + grelha[x-1][y+1] + grelha[x-1][y-1])
    elif x == 0 and y == 0:
        return vetorx[0] + vetorx[1] + vetory[0] + vetory[1] + grelha[x+1][y] + grelha[x][y+1] + grelha[x+1][y+1]
    else:
        return (grelha[x+1][y] + grelha[x-1][y] + grelha[x][y-1] + grelha[x][y+1] 
            + grelha[x+1][y+1] + grelha[x-1][y+1] + grelha[x+1][y-1] + grelha[x-1][y-1])

#Estado inicial do jogo, 3 X 3 tudo a 1
def init(grelha):
    x = centro[0]
    y = centro[1]
    if x == 0 and y == 0:
        zero_zero = 1
        #print("zero:",zero_zero)
        vetorx[0] = 1
        vetory[0] = 1
        vetorx[1] = 1
        vetory[1] = 1
        return And(Equals(grelha[x][y],Int(1)),
                  Equals(grelha[x+1][y],Int(1)),
                  Equals(grelha[x][y+1],Int(1)),
                  Equals(grelha[x+1][y+1],Int(1))
                  )
    #elif x == 0:
    #    vetorx[x-1] = 1
    #    vetorx[x] = 1
    #    vetorx[x+1] = 1
    #    return And(Equals(grelha[x][y],Int(1)),
    #              Equals(grelha[x+1][y],Int(1)),
    #              Equals(grelha[x-1][y],Int(1)),
    #              Equals(grelha[x][y+1],Int(1)),
    #              Equals(grelha[x+1][y+1],Int(1)),
    #              Equals(grelha[x-1][y+1],Int(1))
    #              )
    else:
        return And(Equals(grelha[x][y],Int(1)),
                  Equals(grelha[x+1][y],Int(1)),
                  Equals(grelha[x-1][y],Int(1)),
                  Equals(grelha[x][y-1],Int(1)),
                  Equals(grelha[x][y+1],Int(1)),
                  Equals(grelha[x+1][y+1],Int(1)),
                  Equals(grelha[x-1][y+1],Int(1)),
                  Equals(grelha[x+1][y-1],Int(1)),
                  Equals(grelha[x-1][y-1],Int(1)))


# transições do estado da grelha atual para a próxima
def trans(curr,prox):
    for i in range(N):
        for j in range(N):
            t0 = And(Equals(curr[i][j],Int(0)),
                     Equals(vizinhanca((i,j),curr), Int(3)),
                     Equals(prox[i][j],Int(1))
                    )

            t1 = And(Equals(curr[i][j],Int(1)),
                     Or(Equals(vizinhanca((i,j),curr), Int(3)), Equals(vizinhanca((i,j),curr), Int(2))),
                     Equals(prox[i][j],Int(1))
                    )

            t2 = And(Equals(curr[i][j],Int(1)),
                     Not(Or(Equals(vizinhanca((i,j),curr), Int(3)), Equals(vizinhanca((i,j),curr), Int(2)))),
                     Equals(prox[i][j],Int(0))
                    )

            t3 = And(Equals(curr[i][j],Int(0)),
                     Not(Or(Equals(vizinhanca((i,j),curr), Int(3)), Equals(vizinhanca((i,j),curr), Int(2)))),
                     Equals(prox[i][j],Int(0))
                    )
    
    return Or(t0,t1,t2,t3)
    

### Jogo

Declaração das grelhas do jogo, estado do início e transições.

In [3]:
with Solver(name="z3") as s:
    
    grelhas = []
    for i in range(N):
        grelhas.append([[declare(i,j) for j in range(N)] for i in range(N)])
    #for i in range(N):
    #    grelha.append([])
    #    for j in range(N):
    #        grelha[i].append(declare(i,j))
    #grelha = [[declare(i,j) for j in range(N)] for i in range(N)]    
    print("Campo do jogo:\n", grelhas[0], "\n")
    
    #s.add_assertion(init(grelhas[0],zero_zero,vetorx,vetory))
    s.add_assertion(init(grelhas[0]))
    
    for i in range(N-1):
            s.add_assertion(trans(grelhas[i], grelhas[i+1]))
    
    if s.solve():
        print("Inicio: \n")
        for grelha in grelhas:
            #print(s.get_value(grelhas[g][0][0]))
            for i in range(N):
                if i == 0:
                    print(zero_zero, end = "   ") # constante de zero nao muda para 1 se centro = (0,0)
                    for j in range(N):
                        print(vetorx[j], end = " ")
                    print()
                    print("------------------------")
                print(vetory[i], end = " | ")
                for j in range(N):
                    #print(grelhas[g][i][j], end = " ")
                    print(s.get_value(grelha[i][j]) , end = " ")
                print()
            print("\ntransicao\n")

Campo do jogo:
 [[i0j0, i0j1, i0j2, i0j3, i0j4, i0j5, i0j6], [i1j0, i1j1, i1j2, i1j3, i1j4, i1j5, i1j6], [i2j0, i2j1, i2j2, i2j3, i2j4, i2j5, i2j6], [i3j0, i3j1, i3j2, i3j3, i3j4, i3j5, i3j6], [i4j0, i4j1, i4j2, i4j3, i4j4, i4j5, i4j6], [i5j0, i5j1, i5j2, i5j3, i5j4, i5j5, i5j6], [i6j0, i6j1, i6j2, i6j3, i6j4, i6j5, i6j6]] 

Inicio: 

0   1 0 0 0 0 0 0 
------------------------
0 | 0 0 0 0 0 0 0 
0 | 0 0 0 0 0 0 0 
1 | 0 0 1 1 1 0 0 
1 | 0 0 1 1 1 0 0 
1 | 0 0 1 1 1 0 0 
0 | 0 0 0 0 0 0 3 
0 | 0 0 0 0 0 0 1 

transicao

0   1 0 0 0 0 0 0 
------------------------
0 | 0 0 0 0 0 0 0 
0 | 0 0 0 0 0 0 0 
1 | 0 0 1 1 1 0 0 
1 | 0 0 1 1 1 0 0 
1 | 0 0 1 1 1 0 0 
0 | 0 0 0 0 0 0 3 
0 | 0 0 0 0 0 0 1 

transicao

0   1 0 0 0 0 0 0 
------------------------
0 | 0 0 0 0 0 0 0 
0 | 0 0 0 0 0 0 0 
1 | 0 0 1 1 1 0 0 
1 | 0 0 1 1 1 0 0 
1 | 0 0 1 1 1 0 0 
0 | 0 0 0 0 0 0 3 
0 | 0 0 0 0 0 0 1 

transicao

0   1 0 0 0 0 0 0 
------------------------
0 | 0 0 0 0 0 0 0 
0 | 0 0 0 0 0 0 0 
1 | 0 0 1 1 1 